In [ ]:
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from cellpose import models, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# call logger_setup to have output of cellpose written
# from cellpose.io import logger_setup
# from cellpose import utils

from skimage import io
import tqdm
import napari
import pandas as pd

import zarr
from dask import array as da

In [ ]:
def normalization_two_values(arr, lower, upper):
    """
    Normalize array so that the lower values to be 0 and upper values to be 1.
    """
    return (arr - lower) / (upper - lower)

In [ ]:
# path to dataset and model
dataset_folder = "/mnt/ampa02_data01/gabacoll/shared/Yuchen/model_training/crops"
aug_folder = os.path.join(dataset_folder, 'augment')
train_folder = os.path.join(aug_folder,'training')
models_path = os.path.join(train_folder,'models')

models_file = os.listdir(models_path); models_file.sort()
model_path = os.path.join(train_folder,'models',models_file[-1])

model = models.CellposeModel(gpu=use_GPU, pretrained_model=model_path)

In [ ]:
# image path
n5_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5' # zarr with pyramid resolution
normalization_metadata = '/mnt/ampa02_data01/tmurakami/model_training/norm_values.pkl' # Or None

# create Zarr file object
img_zarr = zarr.open(store=zarr.N5Store(n5_path), mode='r')
voxel_size = (2.0,1.3,1.3)

corner_positions = [1783,2093,2955]
crop_size = [128,512,512]
segment_chan = 1
reference_chan = 3
auto_diam = False # Cellpose automatic diameter estimation.
min_size = 40

# theoretically, anisotropy parameter affects the accuracy. However in practice, changing this values to be the exact voxel ratio does not significantly add accuracy. 
# this may be because of the non-isotropic PSF of light-sheet. 
anisotropy = voxel_size[1]/voxel_size[0] 

# Channel parameters which were used during the training.
Training_channel = 2 # I do not know but the cellpose see the images as KRGB. If the color is green, set it to 2.
Second_training_channel = 1

# load images according to the input parameters.
n5_setups = list(img_zarr.keys())
img_ref = img_zarr[n5_setups[reference_chan]]['timepoint0']['s0']
img_ref_ = img_ref[tuple(slice(i,i+j) for i,j in zip(corner_positions, crop_size))]

img = img_zarr[n5_setups[segment_chan]]['timepoint0']['s0']
img_ = img[tuple(slice(i,i+j) for i,j in zip(corner_positions, crop_size))]

if normalization_metadata is not None:
    norm_info = pd.read_pickle(normalization_metadata)
    img_ref_ = normalization_two_values(img_ref_.astype(float), norm_info[n5_path][reference_chan]['lower'], norm_info[n5_path][reference_chan]['upper'])
    img_ = normalization_two_values(img_.astype(float), norm_info[n5_path][segment_chan]['lower'], norm_info[n5_path][segment_chan]['upper'])
    # img_ = normalization_two_values(img_.astype(float), norm_info[n5_path][segment_chan]['lower'], 5000)

imgs = np.stack([img_ref_,img_])

In [ ]:
# Prediction
if normalization_metadata is not None:
    if ~auto_diam:
        # with diameter parameter provided without Cellpose normalization
        %time masks, flows, styles  = model.eval(imgs, channels=[Training_channel,Second_training_channel], normalize=False, z_axis=1, diameter=model.diam_mean, do_3D=True, min_size=min_size, progress=True, anisotropy=anisotropy)
    else:
        %time masks, flows, styles  = model.eval(imgs, channels=[Training_channel,Second_training_channel], normalize=False, z_axis=1, diameter=None, do_3D=True, min_size=min_size, progress=True, anisotropy=anisotropy)
else:
    if ~auto_diam:
        # with diameter parameter provided 
        %time masks, flows, styles  = model.eval(imgs, channels=[Training_channel,Second_training_channel], z_axis=1, diameter=model.diam_mean, do_3D=True, min_size=min_size, progress=True, anisotropy=anisotropy)
    else:
        # without diameter
        %time masks, flows, styles = model.eval(imgs, channels=[Training_channel,Second_training_channel], z_axis=1, diameter=None, do_3D=True, min_size=min_size, progress=True, anisotropy=anisotropy)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(imgs, channel_axis=0, name='image01', blending='additive')
viewer.add_labels(masks)